In [2]:
#1. what is the diff in percentage between avg yellow cards per game in the cup and champions between df and fw players? 

queryResult("""
WITH 
    total AS (
    SELECT
    squad.position,
    CAST(SUM(cup_stats.cup_yellow) AS float) AS cup_yellow,
    CAST(SUM(games_played.cup) AS float) AS cup_played,
    CAST(SUM(champions_stats.champions_yellow) AS float) AS champions_yellow,
    CAST(SUM(games_played.champions) AS float) AS champions_played
FROM
    squad
LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
LEFT JOIN champions_stats ON squad.player_id = champions_stats.player_id
LEFT JOIN games_played ON squad.player_id = games_played.player_id
WHERE
    games_played.cup >0
    AND position = 'df' OR position = 'fw'
GROUP BY
    squad.position)
SELECT 
    position,
    cup_yellow/cup_played AS avg_yellow_cup,
    champions_yellow/champions_played AS avg_yellow_champions
FROM 
    total
""")

,position,avg_yellow_cup,avg_yellow_champions
0,df,0.156250,0.162791
1,fw,0.185185,0.054054


In [3]:
#2. Per player calculate the avg percentage of games played. (54 are the numbers of games)

queryResult("""
SELECT
    squad.player_id,
    CAST(SUM(games_played.supercup+games_played.league+games_played.champions+games_played.cup) AS float)/54 AS percentage
FROM
    squad
LEFT JOIN games_played ON squad.player_id = games_played.player_id
WHERE
    games_played.cup >0
GROUP BY
    squad.player_id
ORDER BY
    percentage DESC
""")

,player_id,percentage
0,19,0.962963
1,15,0.944444
2,28,0.944444
3,14,0.925926
4,12,0.907407
5,9,0.888889
6,29,0.888889
7,24,0.814815
8,1,0.777778
9,25,0.777778


In [4]:
#3. for each competition return a list of players that didn't get any yellow or red cards

queryResult("""
SELECT
    squad.player_id,
    SUM(supercup_stats.supercup_yellow) + SUM(supercup_stats.supercup_red) AS cards_supercup,
    SUM(league_stats.league_yellow) + SUM(league_stats.league_red) AS cards_league,
    SUM(champions_stats.champions_yellow) + SUM(champions_stats.champions_red) AS cards_champions,
    SUM(cup_stats.cup_yellow) + SUM(cup_stats.cup_red) AS cards_cup
FROM
    squad
LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
LEFT JOIN champions_stats ON squad.player_id = champions_stats.player_id
LEFT JOIN league_stats ON squad.player_id = league_stats.player_id
LEFT JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
GROUP BY
    squad.player_id
HAVING
    (SUM(cup_stats.cup_yellow) + SUM(cup_stats.cup_red)) = 0 OR (SUM(league_stats.league_yellow) + SUM(league_stats.league_red)) = 0
    OR (SUM(champions_stats.champions_yellow) + SUM(champions_stats.champions_red)) = 0 OR (SUM(cup_stats.cup_yellow) + SUM(cup_stats.cup_red)) = 0
""")


,player_id,cards_supercup,cards_league,cards_champions,cards_cup
0,1,0,1,0,0
1,2,0,0,0,0
2,3,0,0,0,0
3,4,0,0,0,0
4,5,0,2,1,0
5,6,0,1,0,0
6,7,0,3,1,0
7,8,0,3,0,0
8,9,1,9,2,0
9,10,0,1,1,0


In [5]:
#4. from each position please return the player that played the most in the cup game and had at least one yellow card

queryResult("""
WITH total AS (
SELECT
    squad.position AS position,
    squad.player_id AS player_id,
    cup_stats.cup_minutes AS cup_minutes,
    cup_stats.cup_yellow AS cup_yellow,
    ROW_NUMBER() OVER (PARTITION BY squad.position ORDER BY cup_stats.cup_minutes DESC) RN
FROM
    squad
LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id

WHERE 
    cup_yellow > 0)
SELECT
    position,
    player_id,
    cup_minutes,
    cup_yellow
FROM
    total
WHERE RN = 1                           
""")


,position,player_id,cup_minutes,cup_yellow
0,df,11,465,1
1,fw,29,510,2
2,mf,17,162,1


In [7]:
#5. What is the average of yellow cards per position? (The share of each position)

queryResult("""
WITH total AS (
SELECT
    SUM(supercup_stats.supercup_yellow+league_stats.league_yellow+champions_stats.champions_yellow+cup_stats.cup_yellow) AS yellow
FROM
    squad
LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
LEFT JOIN champions_stats ON squad.player_id = champions_stats.player_id
LEFT JOIN league_stats ON squad.player_id = league_stats.player_id
LEFT JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
)
SELECT
    DISTINCT(squad.position) AS position,
    SUM(supercup_stats.supercup_yellow+league_stats.league_yellow+champions_stats.champions_yellow+cup_stats.cup_yellow)/AVG(total.yellow) AS share
FROM
    total, squad
LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
LEFT JOIN champions_stats ON squad.player_id = champions_stats.player_id
LEFT JOIN league_stats ON squad.player_id = league_stats.player_id
LEFT JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
GROUP BY
    position
ORDER BY
    share DESC
""")

,position,share
0,df,0.489362
1,mf,0.255319
2,fw,0.244681
3,gk,0.010638


In [64]:
#6. Who are the top 3 players with the best assists per minute record? (How often they make an assist – 3 assists in 900 minutes means assist every 300 minutes)

queryResult("""
WITH
    total AS (
    SELECT
        squad.player_id,
        SUM(supercup_stats.supercup_assists+league_stats.league_assists+champions_stats.champions_assists+cup_stats.cup_assists) AS total_assists,
        SUM(supercup_stats.supercup_minutes+league_stats.league_minutes+champions_stats.champions_minutes+cup_stats.cup_minutes) AS total_minutes
    FROM
        squad LEFT JOIN games_played ON games_played.player_id = squad.player_id
        LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
        LEFT JOIN champions_stats ON squad.player_id = champions_stats.player_id
        LEFT JOIN league_stats ON squad.player_id = league_stats.player_id
        LEFT JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
    GROUP BY
        squad.player_id
        )
SELECT
    player_id,
    SUM(total_minutes/total_assists) AS assists_per_minute
FROM
    total
WHERE
    total_assists > 0 
GROUP BY
    player_id
ORDER BY
    assists_per_minute ASC
LIMIT 
    3;    
""")

,player_id,assists_per_minute
0,22,149.0
1,20,224.0
2,12,279.0


In [212]:
#7. Each normal game time consists of 90 minutes, what is the share of each player playing time in the league and cup from his played games? (Hint: it's ok to get slightly more then 1 in ratio)
#Example: if a player played 1000 minutes and took part in 12 games then his share is 0.92 (1000 / (12*90) )


queryResult("""
WITH total AS (
    SELECT
        squad.player_id AS player_id,
        CAST((SUM(league_stats.league_minutes+cup_stats.cup_minutes)) AS float(2))/ CAST((SUM(games_played.league+games_played.cup)*90) AS float(2)) AS share
    FROM
        squad LEFT JOIN games_played ON games_played.player_id = squad.player_id
        LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
        LEFT JOIN league_stats ON squad.player_id = league_stats.player_id
    WHERE 
        games_played.league > 0 AND games_played.cup > 0
    GROUP BY
        squad.player_id
    ORDER BY
        share DESC)
SELECT 
     player_id,
     CAST(share AS decimal(10,2)) 
FROM
    total
""") 

,player_id,share
0,2,1.04
1,1,1.02
2,15,0.97
3,12,0.96
4,29,0.96
5,7,0.94
6,9,0.85
7,28,0.84
8,11,0.82
9,14,0.79


In [50]:
#8. A forward (fw) is considered to have a good season if he played at least 1500 minutes , took part in at least 18 games and got a minimum of 9 goals + assists.
#How many forwards like that do we have?

queryResult("""
WITH
    total AS (
    SELECT
        squad.player_id,
        squad.position,
        SUM(games_played.league+games_played.champions+games_played.cup+games_played.supercup) AS total_games,
        SUM(supercup_stats.supercup_assists+league_stats.league_assists+champions_stats.champions_assists+cup_stats.cup_assists+supercup_stats.supercup_goals+league_stats.league_goals+champions_stats.champions_goals+cup_stats.cup_goals) AS total_goalsassists,
        SUM(supercup_stats.supercup_minutes+league_stats.league_minutes+champions_stats.champions_minutes+cup_stats.cup_minutes) AS total_minutes
    FROM
        squad LEFT JOIN games_played ON games_played.player_id = squad.player_id
        LEFT JOIN cup_stats ON squad.player_id = cup_stats.player_id
        LEFT JOIN champions_stats ON squad.player_id = champions_stats.player_id
        LEFT JOIN league_stats ON squad.player_id = league_stats.player_id
        LEFT JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
    WHERE
        squad.position = 'fw'
    GROUP BY
        squad.player_id,
        squad.position
        )
SELECT
    player_id,
    SUM(total_games) AS total_games,
    SUM(total_goalsassists) AS total_goalsassists,
    SUM(total_minutes) AS total_minutes
FROM
    total
WHERE
    total_games >= 18 AND total_minutes >= 1500 AND total_goalsassists >= 9
GROUP BY
    player_id
""")

,player_id,total_games,total_goalsassists,total_minutes
0,27,40.0,11.0,1716.0
1,29,48.0,52.0,4192.0
2,28,51.0,33.0,3903.0
3,24,44.0,16.0,2699.0
